In [1]:
from bs4 import BeautifulSoup
import numpy as np
import os
import scipy.stats as s

In [2]:
# choose arguments
average_last_n = 2 # number of final RLCT values averaged from 1 epoch

In [3]:
# converts javascript code from plotly plot in html to numpy arrays
def jscript_to_array(js_code, start_string, end_string):
    start_len = len(start_string)

    start_index = js_code.find(start_string)
    if start_index != -1: 
        start_index += start_len
        rlct_end_index = js_code.find(end_string, start_index)
        array_string = js_code[start_index:rlct_end_index]
        numpy_array = np.array(eval(array_string))
        return numpy_array
    return np.array([])

In [43]:
def data_from_file(directory, filename, average_last_n):
    html_file_path = directory + '/' + filename

    # Load the HTML file
    with open(html_file_path, "r") as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, "html.parser")

    # Find all <script> tags
    script_tags = soup.find_all("script")

    # helper string initialisation
    sgd_rlct_start_string = 'sgd RLCT","x":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],"y":'
    ngd_rlct_start_string = 'ngd RLCT","x":[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],"y":'
    rlct_end_string = ',"type":"scatter"'

    sgd_matrix = []
    ngd_matrix = []

    # Iterate over each <script> tag
    for script_tag in script_tags:
        # Extract the JavaScript code
        js_code = script_tag.string

        # Check if the script contains Plotly initialization code
        if js_code and "Plotly.newPlot" in js_code:
            # sgd
            rlct_array = jscript_to_array(js_code, sgd_rlct_start_string, rlct_end_string)
            if (rlct_array.size > 0):
                sgd_matrix.append(rlct_array)


            #ngd
            rlct_array = jscript_to_array(js_code, ngd_rlct_start_string, rlct_end_string)
            if (rlct_array.size > 0):
                ngd_matrix.append(rlct_array)

    sgd_matrix = np.array(sgd_matrix)
    ngd_matrix = np.array(ngd_matrix)

    sgd_matrix_average = sgd_matrix[:,-average_last_n:].mean(axis = 1)
    ngd_matrix_average = ngd_matrix[:,-average_last_n:].mean(axis = 1)

    return sgd_matrix_average, ngd_matrix_average

    


In [44]:
# Get the experiment directory
directory = "ngd_sgd"
count = 0

sgd_rlct_values = []
ngd_rlct_values = []

# Loop through the files in the current working directory
for filename in os.listdir(directory):
    # Check if the filename contains the word "seed"
    if "seed" in filename:
        count += 1
        sgd_matrix_average, ngd_matrix_average = data_from_file(directory, filename, average_last_n)
        sgd_rlct_values.append(sgd_matrix_average)
        ngd_rlct_values.append(ngd_matrix_average)

print("number of files: ", count)

number of files:  30


In [45]:
# experiment format: [rlct1, rlct2, rlct3, rlct4, rlct5, rlct6]
# RLCTs obtained from the following settings:
# "hidden_nodes" : [64, 128, 256, 64, 128, 256], "hidden_layers" : [1, 1, 1, 2, 2, 2],

# calculate the std and mean of the 2 distrbutions
sgd_rlct_values = np.array(sgd_rlct_values)
ngd_rlct_values = np.array(ngd_rlct_values)

sgd_rlct_std = np.std(sgd_rlct_values, axis=0)
ngd_rlct_std = np.std(ngd_rlct_values, axis=0)

sgd_rlct_mean = np.mean(sgd_rlct_values, axis=0)
ngd_rlct_mean = np.mean(ngd_rlct_values, axis=0)

print("sgd rlct std: ", sgd_rlct_std)
print("ngd rlct std: ", ngd_rlct_std)
print()
print("sgd rlct mean: ", sgd_rlct_mean)
print("ngd rlct mean: ", ngd_rlct_mean)

sgd rlct std:  [0.58972984 0.6248595  0.72181244 0.82955899 0.74946956 0.92028196]
ngd rlct std:  [0.83250222 0.77634998 0.85134888 1.0478341  1.4762508  0.98220954]

sgd rlct mean:  [16.30601522 18.84313513 23.24316651 25.46965288 32.85987771 43.84351247]
ngd rlct mean:  [23.70035092 31.36502829 36.4120868  31.77519293 42.4361625  55.91884823]


In [46]:
def preety_print():
    hidden_nodes = [64, 128, 256, 64, 128, 256]
    hidden_layers = [1, 1, 1, 2, 2, 2]

    for i in range(len(hidden_nodes)):
        print(f"\n{hidden_nodes[i]} hidden node/s | {hidden_layers[i]} hidden layer/s")
        print("\nsgd rlct std: ", sgd_rlct_std[i])
        print("ngd rlct std: ", ngd_rlct_std[i])
        print("\nsgd rlct mean: ", sgd_rlct_mean[i])
        print("ngd rlct mean: ", ngd_rlct_mean[i])
        print("\n", "-"*40)

preety_print()


64 hidden node/s | 1 hidden layer/s

sgd rlct std:  0.5897298400468453
ngd rlct std:  0.832502220840553

sgd rlct mean:  16.30601522127787
ngd rlct mean:  23.700350920359295

 ----------------------------------------

128 hidden node/s | 1 hidden layer/s

sgd rlct std:  0.6248595031612819
ngd rlct std:  0.7763499845575881

sgd rlct mean:  18.843135134379068
ngd rlct mean:  31.365028285980223

 ----------------------------------------

256 hidden node/s | 1 hidden layer/s

sgd rlct std:  0.7218124397779772
ngd rlct std:  0.8513488841291114

sgd rlct mean:  23.24316651026408
ngd rlct mean:  36.41208680470785

 ----------------------------------------

64 hidden node/s | 2 hidden layer/s

sgd rlct std:  0.8295589892312767
ngd rlct std:  1.0478341025021805

sgd rlct mean:  25.469652875264487
ngd rlct mean:  31.77519292831421

 ----------------------------------------

128 hidden node/s | 2 hidden layer/s

sgd rlct std:  0.7494695626319364
ngd rlct std:  1.476250803988563

sgd rlct mean:  

In [47]:
# hypothesis test: rlct_ngd > rlct_sgd
print("Hypothesis Test")
print("\nNull Hypothesis: rlct_ngd is equal to rlct_sgd")
print("Alternative Hypothesis: rlct_ngd is greater than rlct_sgd")
print("\n", "-"*40)
# test 1: two tailed Welch's t-test 
print("\ntest 1: two-tailed Welch's t-test\n")
print("This test compares if NGD's distribution is higher than SGD's distribution\n")
result = s.ttest_ind(sgd_rlct_values, ngd_rlct_values, axis=0, equal_var=False, alternative="two-sided")
print("Welch's t-test pvalues: ", result.pvalue)
print("\nConclusion: rlct_ngd is greater than rlct_sgd")
print("\n", "-"*40)

# test 2: one-tailed test using seed 1
print("\ntest 2: one-tailed z-test uing seed 1\n")
print("This test checks if NGD values for seed 1 are higher than SGD\n")
z_score = (np.array([22.8, 31.3, 36.8, 33, 43.5, 57.4]) - sgd_rlct_mean)/sgd_rlct_std
print("z scores: ", z_score)
print("p values for ngd seed1: ",1 - s.norm.cdf(z_score))
print("\nConclusion: rlct_ngd for seed 1 is greater than rlct_sgd")

Hypothesis Test

Null Hypothesis: rlct_ngd is equal to rlct_sgd
Alternative Hypothesis: rlct_ngd is greater than rlct_sgd

 ----------------------------------------

test 1: one tailed Welch's t-test

This test compares if NGD's distribution is higher than SGD's distribution

Welch's t-test pvalues:  [1.25368769e-40 2.68355304e-55 1.54212240e-54 2.09295033e-32
 1.89996977e-31 1.05624311e-48]

Conclusion: rlct_ngd is greater than rlct_sgd

 ----------------------------------------

test 2: one-tailed z-test uing seed 1

This test checks if NGD values for seed 1 are higher than SGD

z scores:  [11.01179614 19.93546518 18.78165676  9.07753062 14.19687045 14.73079783]
p values for ngd seed1:  [0. 0. 0. 0. 0. 0.]

Conclusion: rlct_ngd for seed 1 is greater than rlct_sgd


In [14]:
def valLoss_data_from_file(directory, filename, average_last_n):
    html_file_path = directory + '/' + filename

    # Load the HTML file
    with open(html_file_path, "r") as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, "html.parser")

    # Find all <script> tags
    script_tags = soup.find_all("script")

    # helper string initialisation
    sgd_loss_start_string = 'sgd Validation","x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],"y":'
    ngd_loss_start_string = 'ngd Validation","x":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],"y":'
    loss_end_string = ',"type":"scatter"'

    sgd_matrix = []
    ngd_matrix = []

    # Iterate over each <script> tag
    for script_tag in script_tags:
        # Extract the JavaScript code
        js_code = script_tag.string

        # Check if the script contains Plotly initialization code
        if js_code and "Plotly.newPlot" in js_code:
            # sgd
            loss_array = jscript_to_array(js_code, sgd_loss_start_string, loss_end_string)
            if (loss_array.size > 0):
                sgd_matrix.append(loss_array)

            #ngd
            loss_array = jscript_to_array(js_code, ngd_loss_start_string, loss_end_string)
            if (loss_array.size > 0):
                ngd_matrix.append(loss_array)

    sgd_matrix = np.array(sgd_matrix)
    ngd_matrix = np.array(ngd_matrix)

    sgd_matrix_average = sgd_matrix[:,-average_last_n:].mean(axis = 1)
    ngd_matrix_average = ngd_matrix[:,-average_last_n:].mean(axis = 1)

    return sgd_matrix_average, ngd_matrix_average


In [16]:
# Get the experiment directory
directory = "ngd_sgd"
count = 0

sgd_loss_values = []
ngd_loss_values = []

# Loop through the files in the current working directory
for filename in os.listdir(directory):
    # Check if the filename contains the word "seed"
    if "seed" in filename:
        count += 1
        sgd_matrix_average, ngd_matrix_average = valLoss_data_from_file(directory, filename, average_last_n)
        sgd_loss_values.append(sgd_matrix_average)
        ngd_loss_values.append(ngd_matrix_average)

print("number of files: ", count)

number of files:  30


In [18]:
# experiment format: [rlct1, rlct2, rlct3, rlct4, rlct5, rlct6]
# RLCTs obtained from the following settings:
# "hidden_nodes" : [64, 128, 256, 64, 128, 256], "hidden_layers" : [1, 1, 1, 2, 2, 2],

# calculate the std and mean of the 2 distrbutions
sgd_loss_values = np.array(sgd_loss_values)
ngd_loss_values = np.array(ngd_loss_values)

sgd_loss_std = np.std(sgd_loss_values, axis=0)
ngd_loss_std = np.std(ngd_loss_values, axis=0)

sgd_loss_mean = np.mean(sgd_loss_values, axis=0)
ngd_loss_mean = np.mean(ngd_loss_values, axis=0)

print("sgd validation loss std: ", sgd_loss_std)
print("ngd validation loss std: ", ngd_loss_std)
print()
print("sgd validation loss mean: ", sgd_loss_mean)
print("ngd validation loss mean: ", ngd_loss_mean)

sgd validation loss std:  [0.00299866 0.00251522 0.00145078 0.00531064 0.00298234 0.00244082]
ngd validation loss std:  [0.0055832  0.00393159 0.00168889 0.00782029 0.00252469 0.00137107]

sgd validation loss mean:  [0.0793424  0.06668437 0.06020073 0.09059092 0.07461914 0.06621484]
ngd validation loss mean:  [0.12262951 0.08274211 0.06415949 0.13063709 0.06936007 0.06170833]


In [20]:
def loss_preety_print():
    hidden_nodes = [64, 128, 256, 64, 128, 256]
    hidden_layers = [1, 1, 1, 2, 2, 2]

    for i in range(len(hidden_nodes)):
        print(f"\n{hidden_nodes[i]} hidden node/s | {hidden_layers[i]} hidden layer/s")
        print("\nsgd validation loss std: ", sgd_loss_std[i])
        print("ngd validation loss std: ", ngd_loss_std[i])
        print("\nsgd validation loss mean: ", sgd_loss_mean[i])
        print("ngd validation loss mean: ", ngd_loss_mean[i])
        print("\n", "-"*40)

loss_preety_print()


64 hidden node/s | 1 hidden layer/s

sgd validation loss std:  0.0029986601833465257
ngd validation loss std:  0.005583198553547572

sgd validation loss mean:  0.07934240114578633
ngd validation loss mean:  0.12262951286639893

 ----------------------------------------

128 hidden node/s | 1 hidden layer/s

sgd validation loss std:  0.0025152200285470877
ngd validation loss std:  0.003931587514038888

sgd validation loss mean:  0.0666843683116012
ngd validation loss mean:  0.08274211077140407

 ----------------------------------------

256 hidden node/s | 1 hidden layer/s

sgd validation loss std:  0.0014507797386649655
ngd validation loss std:  0.0016888898617883215

sgd validation loss mean:  0.06020073245990523
ngd validation loss mean:  0.0641594887455129

 ----------------------------------------

64 hidden node/s | 2 hidden layer/s

sgd validation loss std:  0.0053106374420653045
ngd validation loss std:  0.007820290991462114

sgd validation loss mean:  0.09059092267628283
ngd v

In [23]:
# hypothesis test: loss_ngd > loss_sgd
print("Hypothesis Test")
print("\nNull Hypothesis: validation loss_ngd is equal to validation loss_sgd")
print("Alternative Hypothesis: validation loss_ngd is greater than validation loss_sgd")
print("\n", "-"*40)
# test 1: two-tailed Welch's t-test 
print("\ntest 1: two tailed Welch's t-test\n")
print("This test compares if NGD's distribution is higher than SGD's distribution\n")
result = s.ttest_ind(sgd_loss_values, ngd_loss_values, axis=0, equal_var=False, alternative="two-sided")
print("Welch's t-test pvalues: ", result.pvalue)
print("\nConclusion: validation loss_ngd is not always greater than validation loss_sgd")
print("\n", "-"*40)

# test 2: one-tailed test using seed 1
print("\ntest 2: one-tailed z-test uing seed 1\n")
print("This test checks if NGD values for seed 1 are higher than SGD\n")
z_score = (np.array([0.1306, 0.0878, 0.0636, 0.1273, 0.0694, 0.0611]) - sgd_loss_mean)/sgd_loss_std
print("z scores: ", z_score)
print("p values for ngd seed1: ",1 - s.norm.cdf(z_score))
print("\nConclusion: validation loss_ngd for seed 1 is greater than validation loss_sgd")

Hypothesis Test

Null Hypothesis: validation loss_ngd is equal to validation loss_sgd
Alternative Hypothesis: validation loss_ngd is greater than validation loss_sgd

 ----------------------------------------

test 1: one tailed Welch's t-test

This test compares if NGD's distribution is higher than SGD's distribution

Welch's t-test pvalues:  [3.18379549e-35 3.63358866e-24 9.48144641e-14 9.68209733e-29
 9.99999999e-01 1.00000000e+00]

Conclusion: validation loss_ngd is not always greater than validation loss_sgd

 ----------------------------------------

test 2: one-tailed z-test uing seed 1

This test checks if NGD values for seed 1 are higher than SGD

z scores:  [17.09350034  8.39514295  2.34306246  6.91236744 -1.75001376 -2.09554007]
p values for ngd seed1:  [0.00000000e+00 0.00000000e+00 9.56309051e-03 2.38320474e-12
 9.59942031e-01 9.81938495e-01]

Conclusion: validation loss_ngd for seed 1 is greater than validation loss_sgd
